# Data preparation 
##### Bike rides for the past 12 months


### 1. Merge raw data

Merge all the tables into a single dataframe

Each table corresponds to a month of the past year

In [5]:
import pandas as pd
import os
import sqlite3
import geopy.distance

In [6]:
# ride_data is the folder where all the data is stored
cd = os.getcwd()
directory = "{}\\ride_data".format(cd)

In [7]:
# Creates the DF
df = pd.read_csv(directory+"\\divvy_trip (1).csv")

In [8]:
# Loop through the last 11 months of Ride Data and append to the existing df
for file in os.listdir(directory):
    if not directory+"\\divvy_trip (1).csv" == directory+"\\"+file:
        df = df.append(pd.DataFrame(pd.read_csv(directory+"\\"+file)), ignore_index=True)
print('done')

done


In [9]:
# Is there any Null values in the data?
df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    745376
start_station_id      745373
end_station_name      796247
end_station_id        796247
start_lat                  0
start_lng                  0
end_lat                 4716
end_lng                 4716
member_casual              0
dtype: int64

In [10]:
# Remove the Null values
df.dropna(inplace=True)

In [12]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.886875,-87.626030,41.889177,-87.638506,member
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.900998,-87.623752,41.898203,-87.637536,member
2,EFE527AF80B66109,classic_bike,2022-03-23 19:52:02,2022-03-23 19:54:48,Broadway & Berwyn Ave,13109,Broadway & Ridge Ave,15578,41.978353,-87.659753,41.984045,-87.660274,member
3,9F446FD9DEE3F389,classic_bike,2022-03-01 19:12:26,2022-03-01 19:22:14,Wabash Ave & Wacker Pl,TA1307000131,Franklin St & Jackson Blvd,TA1305000025,41.886875,-87.626030,41.877708,-87.635321,member
4,431128AD9AFFEDC0,classic_bike,2022-03-21 18:37:01,2022-03-21 19:19:11,DuSable Lake Shore Dr & North Blvd,LF-005,Loomis St & Jackson Blvd,13206,41.911722,-87.626804,41.877945,-87.662007,member


### 2. Date convertion
* Converting dates into seconds
* New column: weekday >> 0 is Monday, 6 is Sunday
* New column: duration_ns >> end - start in nanoseconds
* Remove ride durations < 1min and > 24hrs

In [14]:
# Convert to datetime and get the weekday
df["started_at"] = pd.to_datetime(df["started_at"], yearfirst=True)
df["ended_at"] = pd.to_datetime(df["ended_at"], yearfirst=True)
df['weekday'] = df['started_at'].dt.dayofweek

In [15]:
# Calculate the ride duration
df['duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds()

In [17]:
# Remove any ride that is too short or too long (outliers)
df = df[df['duration'] >= 60]        # 1 minute
df = df[df['duration'] <= 24*60*60]  # 24 hours

In [18]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4579009 entries, 0 to 5723481
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  weekday             int64         
 14  duration            float64       
dtypes: datetime64[ns](2), float64(5), int64(1), object(7)
memory usage: 2.4 GB


In [19]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,weekday,duration
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.886875,-87.626030,41.889177,-87.638506,member,0,377.0
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.900998,-87.623752,41.898203,-87.637536,member,2,378.0
2,EFE527AF80B66109,classic_bike,2022-03-23 19:52:02,2022-03-23 19:54:48,Broadway & Berwyn Ave,13109,Broadway & Ridge Ave,15578,41.978353,-87.659753,41.984045,-87.660274,member,2,166.0
3,9F446FD9DEE3F389,classic_bike,2022-03-01 19:12:26,2022-03-01 19:22:14,Wabash Ave & Wacker Pl,TA1307000131,Franklin St & Jackson Blvd,TA1305000025,41.886875,-87.626030,41.877708,-87.635321,member,1,588.0
4,431128AD9AFFEDC0,classic_bike,2022-03-21 18:37:01,2022-03-21 19:19:11,DuSable Lake Shore Dr & North Blvd,LF-005,Loomis St & Jackson Blvd,13206,41.911722,-87.626804,41.877945,-87.662007,member,0,2530.0


### 3. Get unique stations

Filter only unique stations


In [87]:
# Check if all the end_stations are in the start_stations
unique_stations = df[~df['end_station_name'].isin(df['start_station_name'])].drop_duplicates('end_station_name')[['end_station_name', 'end_lat', 'end_lng']]
unique_stations.head()

,end_station_name,end_lat,end_lng
2412049,Whipple St & Irving Park Rd,41.95,-87.7


In [88]:
# Creates a unique station dataframe
all_stations = df[['start_station_name', 'start_lat', 'start_lng']].drop_duplicates('start_station_name')
all_stations.head()

,start_station_name,start_lat,start_lng
0,Wabash Ave & Wacker Pl,41.886875,-87.626030
1,Michigan Ave & Oak St,41.900998,-87.623752
2,Broadway & Berwyn Ave,41.978353,-87.659753
4,DuSable Lake Shore Dr & North Blvd,41.911722,-87.626804
5,Bissell St & Armitage Ave,41.918018,-87.652182


In [89]:
# Convert a unique_stations columns name to all_stations columns' name
new_cols = {x: y for x, y in zip(unique_stations.columns, all_stations.columns)}
df_unique = unique_stations.rename(columns=new_cols)

In [90]:
# Append the unique_stations to the all_stations
all_stations = all_stations.append(df_unique).sort_values('start_station_name')

In [91]:
# Create a final columns names for the all_stations

# Remove the 'start_' from columns names
all_col = [col.replace('start_', '') for col in all_stations.columns.to_list()]

# Create dict to replace the old with the new names
new_cols2 = {x: y for x, y in zip(all_stations.columns, all_col)}

# Rename
all_stations.rename(columns=new_cols2, inplace=True)

In [92]:
all_stations.head()

,station_name,lat,lng
17533,2112 W Peterson Ave,41.991174,-87.683578
4177511,351,41.930000,-87.780000
1258,63rd St Beach,41.780911,-87.576324
526,900 W Harrison St,41.874768,-87.649829
2254,Aberdeen St & Jackson Blvd,41.877726,-87.654787


### 4. Stations' distance
Calculate distance using geografical coordanates

In [101]:
# Function to calculate the distance between stations

def station_dist(list_coord): # string list_coord: start lat, start lng, end lat, end lng
    # To list >> To float
    coord = [float(coord) for coord in list_coord.split(" ")]
    # start and end coord must be a tuple of (lat, long)
    dist = geopy.distance.distance((coord[0], coord[1]), (coord[2], coord[3])).m
    return int(round(dist, 0))

In [94]:
# Join the coordinates separeted by space
df['stations_coord'] = df['start_lat'].astype(str) + " " + df['start_lng'].astype(str) + " " + df['end_lat'].astype(str) + " " + df['end_lng'].astype(str)

In [100]:
# It takes time to calculate coordinates into distance. Be patiente.
df['stations_distance_m'] = df.stations_coord.apply(station_dist)

In [103]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,weekday,duration,stations_coord,stations_distance_m
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.886875,-87.626030,41.889177,-87.638506,member,0,377.0,41.886875 -87.62603 41.88917683258 -87.6385057718,1067
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.900998,-87.623752,41.898203,-87.637536,member,2,378.0,41.90099766666667 -87.62375166666666 41.898203...,1185
2,EFE527AF80B66109,classic_bike,2022-03-23 19:52:02,2022-03-23 19:54:48,Broadway & Berwyn Ave,13109,Broadway & Ridge Ave,15578,41.978353,-87.659753,41.984045,-87.660274,member,2,166.0,41.978353 -87.659753 41.9840446107 -87.6602738295,634
3,9F446FD9DEE3F389,classic_bike,2022-03-01 19:12:26,2022-03-01 19:22:14,Wabash Ave & Wacker Pl,TA1307000131,Franklin St & Jackson Blvd,TA1305000025,41.886875,-87.626030,41.877708,-87.635321,member,1,588.0,41.886875 -87.62603 41.8777079559 -87.6353211408,1277
4,431128AD9AFFEDC0,classic_bike,2022-03-21 18:37:01,2022-03-21 19:19:11,DuSable Lake Shore Dr & North Blvd,LF-005,Loomis St & Jackson Blvd,13206,41.911722,-87.626804,41.877945,-87.662007,member,0,2530.0,41.911722 -87.626804 41.877945 -87.662007,4755


## 5. Dataframes and Database creation
#### We are going to separate this data into mulitple tables

* rides
* ride_duration  
* stations 

In [104]:
df_rides = df[['ride_id', 'rideable_type', 'start_station_name',
               'end_station_name', 'stations_distance_m', 'member_casual']]
df_rides

,ride_id,rideable_type,start_station_name,end_station_name,stations_distance_m,member_casual
0,47EC0A7F82E65D52,classic_bike,Wabash Ave & Wacker Pl,Kingsbury St & Kinzie St,1067,member
1,8494861979B0F477,electric_bike,Michigan Ave & Oak St,Orleans St & Chestnut St (NEXT Apts),1185,member
2,EFE527AF80B66109,classic_bike,Broadway & Berwyn Ave,Broadway & Ridge Ave,634,member
3,9F446FD9DEE3F389,classic_bike,Wabash Ave & Wacker Pl,Franklin St & Jackson Blvd,1277,member
4,431128AD9AFFEDC0,classic_bike,DuSable Lake Shore Dr & North Blvd,Loomis St & Jackson Blvd,4755,member
...,...,...,...,...,...,...
5723378,1B88F66E86C094DB,classic_bike,Clark St & Leland Ave,Clark St & Leland Ave,0,member
5723400,E73A038DA647AAFF,docked_bike,Michigan Ave & Oak St,Michigan Ave & Oak St,0,casual
5723421,4AD181F39CCB99ED,classic_bike,Kingsbury St & Kinzie St,Desplaines St & Kinzie St,496,member
5723462,D6AE7BEA1D494E4B,classic_bike,Michigan Ave & Oak St,Michigan Ave & Oak St,0,member


In [105]:
df_ride_duration = df[['ride_id', 'member_casual', 'started_at', 'ended_at', 'duration', 'weekday']]
df_ride_duration

,ride_id,member_casual,started_at,ended_at,duration,weekday
0,47EC0A7F82E65D52,member,2022-03-21 13:45:01,2022-03-21 13:51:18,377.0,0
1,8494861979B0F477,member,2022-03-16 09:37:16,2022-03-16 09:43:34,378.0,2
2,EFE527AF80B66109,member,2022-03-23 19:52:02,2022-03-23 19:54:48,166.0,2
3,9F446FD9DEE3F389,member,2022-03-01 19:12:26,2022-03-01 19:22:14,588.0,1
4,431128AD9AFFEDC0,member,2022-03-21 18:37:01,2022-03-21 19:19:11,2530.0,0
...,...,...,...,...,...,...
5723378,1B88F66E86C094DB,member,2021-07-18 11:51:19,2021-07-18 12:36:24,2705.0,6
5723400,E73A038DA647AAFF,casual,2021-07-23 11:30:06,2021-07-23 12:37:25,4039.0,4
5723421,4AD181F39CCB99ED,member,2021-07-12 17:42:02,2021-07-12 17:45:44,222.0,0
5723462,D6AE7BEA1D494E4B,member,2021-07-17 17:01:24,2021-07-17 17:38:14,2210.0,5


In [106]:
df_stations = all_stations
df_stations

,station_name,lat,lng
17533,2112 W Peterson Ave,41.991174,-87.683578
4177511,351,41.930000,-87.780000
1258,63rd St Beach,41.780911,-87.576324
526,900 W Harrison St,41.874768,-87.649829
2254,Aberdeen St & Jackson Blvd,41.877726,-87.654787
...,...,...,...
9513,Woodlawn Ave & 55th St,41.795264,-87.596471
114836,Woodlawn Ave & 75th St,41.759160,-87.595751
12022,Woodlawn Ave & Lake Park Ave,41.814096,-87.597005
670,Yates Blvd & 75th St,41.758768,-87.566440


### 6. Export to a SQL database

In [107]:
con = sqlite3.connect('divy.db')

In [108]:
# Save the data
df_rides.to_sql("Rides", index=False, con=con)
df_ride_duration.to_sql("Rides_duration", index=False, con=con)
df_stations.to_sql("Stations", index=False, con=con)